In [ ]:
from bs4.element import nonwhitespace_re
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd

base = 'https://courses.illinois.edu/cisapp/explorer/catalog'
year, term, subject, courseNumber = ('',)*4

def get_Soup():
  #creating each different url we need data from
  url = base
  if year != '':
    url += '/' + year
  if term != '':
    url += '/' + term
  if subject != '':
    url += '/' + subject
  if courseNumber != '':
    url += '/' + courseNumber
  url += '.xml'
  page = requests.get(url).text
  return BeautifulSoup(page, "xml")

dictionary = {}

yearArray = get_Soup().findAll('calendarYear')
for currentYear in yearArray:
  year = currentYear.string
  if year != '2023':
    year = ''
    continue
  print(year)
  termArray = get_Soup().findAll('term')
  for currentTerm in termArray:
    term = currentTerm.string.split()[0].lower()
    if term != 'fall':
      term = ''
      continue
    print(term)
    subjectArray = get_Soup().findAll('subject')
    for currentSubject in subjectArray:
      subject = currentSubject.get('id')
      print(subject)
      courseNumberArray = get_Soup().findAll('course')
      for currentCourseNumber in courseNumberArray:
        courseNumber = currentCourseNumber.get('id')
        if dictionary.get(subject + " " + courseNumber) != None:
          continue
        #finally can access all info for a specific class
        soup = get_Soup()
        print(subject + " " + courseNumber + ": " + soup.findAll('label')[0].string)
        description = ""
        prerequisite = ''
        if len(soup.findAll('description')) != 0:
          description_and_prereq = soup.findAll('description')[0].string.split('Prerequisite: ')
          description = description_and_prereq[0]
          #if 'Prerequisite:' isn't in the description or what's after 'Prerequisite' is empty
          if len(description_and_prereq) > 1 and len(description_and_prereq[1].strip()) != 0:
            prerequisite = description_and_prereq[1]

        creditHours = soup.findAll('creditHours')[0].string
        creditHours = creditHours.rstrip(creditHours[-1])

        genEdAttribute = soup.findAll('genEdAttributes')
        listOfGenEdAttributes = ''
        if len(genEdAttribute) == 0:
          listOfGenEdAttributes = ''
        else:
          for attribute in genEdAttribute:
            listOfGenEdAttributes += attribute.string + ","
          listOfGenEdAttributes = listOfGenEdAttributes.rstrip(listOfGenEdAttributes[-1])

        dictionary[subject + " " + courseNumber] = [term, year, subject, courseNumber, description, prerequisite, creditHours, listOfGenEdAttributes]

        #clear values
        courseNumber = ''
      subject = ''
    term = ''

df = pd.DataFrame.from_dict(dictionary)
df = df.transpose()
df.columns = ['Term', 'Year', 'Subject', 'Course Number', 'Description', 'Prerequisite', 'Credit Hours', 'List of Gen Ed Attributes']
df.to_csv('courseData.csv', sep='\t')
print(df)

Streaming output truncated to the last 5000 lines.
GGIS 446: Sustainable Planning Seminar
GGIS 455: Geography of Sub-Saharan Africa
GGIS 459: Ecohydraulics
GGIS 460: Aerial Photo Analysis
GGIS 465: Transportation &Sustainability
GGIS 466: Environmental Policy
GGIS 468: Biological Modeling
GGIS 471: Modern Geographic Thought
GGIS 473: Digital Cartography & Map Design
GGIS 476: Applied GIS to Environ Studies
GGIS 477: Introduction to Remote Sensing
GGIS 478: Techniques of Remote Sensing
GGIS 479: Advanced Topics in GIS
GGIS 480: Principles of Geographic Information Science
GGIS 482: Challenges of Sustainability
GGIS 483: Urban Geography
GGIS 484: Cities, Crime, and Space
GGIS 489: Programming for GIS
GGIS 491: Research in Geography
GGIS 495: Advanced Topics in Geography
GGIS 496: Climate & Social Vulnerability
GGIS 507: High-Performance Geospatial Computing
GGIS 517: Geospatial Visualization & Visual Analytics
GGIS 520: Political Ecology
GGIS 527: Geospatial Artificial Intelligence & Mac

In [ ]:
from bs4.element import nonwhitespace_re
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd

base = 'https://1010labs.org/gpa?course='
year, term, subject, courseNumber = ('',)*4

def get_Soup():
  #creating each different url we need data from
  url = base
  if subject != '':
    url += subject + '+'
  if courseNumber != '':
    url += courseNumber + '&semester=All' #or can pull 2021 fall data
  page = requests.get(url).text
  return BeautifulSoup(page, "xml")



#https://raw.githubusercontent.com/chin123/gpa/master/uiuc-gpa-dataset.csv

In [ ]:
from bs4.element import nonwhitespace_re
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd

url = 'https://students.grainger.illinois.edu/club_signup?view=all&group_type=14659&category_tags='
page = requests.get(url).text
soup = BeautifulSoup(page, "html")

rsoNames = []
rsoWebsites = []

nameWebsiteData = soup.findAll('a', {'target':'_blank'})
for i in nameWebsiteData:
  if (i.text.strip() != '' and i.text.strip() != 'Website'):
    rsoNames.append(i.text.strip())
    rsoWebsites.append(i['href'].strip())

rsoMissions = []
keep = True
missionData = soup.findAll('p', {'class':'noOutlineOnFocus'})
missions = []
for i in missionData:
  if (keep):
    missions.append(i)
    rsoMissions.append(i.text.replace('Mission', '').strip())
  keep = not keep

rsoInfo = [rsoNames, rsoWebsites, rsoMissions]
df = pd.DataFrame(rsoInfo)
df = df.transpose()
df.columns = ['Name', 'Website', 'Mission']
df.to_csv('rsoData.csv', sep='\t')